# Import libraries

In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from IPython.display import Markdown

# Step 1: Load the data

In [52]:
df = pd.read_excel('Online Retail.xlsx')

# Step 2: Understand the data

In [54]:
print("First 5 rows:", df.head())

First 5 rows:   InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  UnitPrice  CustomerID         Country  
0 2010-12-01 08:26:00       2.55     17850.0  United Kingdom  
1 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
2 2010-12-01 08:26:00       2.75     17850.0  United Kingdom  
3 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
4 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  


In [55]:
print("\nData Info:")
df.info()


Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [56]:
print("\nDescriptive Statistics:\n", df.describe())


Descriptive Statistics:
             Quantity                    InvoiceDate      UnitPrice  \
count  541909.000000                         541909  541909.000000   
mean        9.552250  2011-07-04 13:34:57.156386048       4.611114   
min    -80995.000000            2010-12-01 08:26:00  -11062.060000   
25%         1.000000            2011-03-28 11:34:00       1.250000   
50%         3.000000            2011-07-19 17:17:00       2.080000   
75%        10.000000            2011-10-19 11:27:00       4.130000   
max     80995.000000            2011-12-09 12:50:00   38970.000000   
std       218.081158                            NaN      96.759853   

          CustomerID  
count  406829.000000  
mean    15287.690570  
min     12346.000000  
25%     13953.000000  
50%     15152.000000  
75%     16791.000000  
max     18287.000000  
std      1713.600303  


In [57]:
print("\nMissing Values:\n", df.isnull().sum())


Missing Values:
 InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64


# Step 3: Clean the data

In [59]:
# Drop rows with missing CustomerID or Description
df.dropna(subset=['CustomerID', 'Description'], inplace=True)


In [60]:
# Remove duplicates
df.drop_duplicates(inplace=True)

In [61]:
# Remove negative or zero Quantity or UnitPrice
df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]


In [62]:
# Remove cancelled orders (InvoiceNo starting with 'C')
df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]


In [63]:
df['CustomerID'] = df['CustomerID'].astype(int)


# Step 4: Feature Engineering 

In [65]:
# TotalAmount per transaction
df['TotalAmount'] = df['Quantity'] * df['UnitPrice']

In [66]:
# Aggregate data by CustomerID
customer_df = df.groupby('CustomerID').agg({
    'InvoiceNo': 'nunique',
    'Quantity': 'sum',
    'TotalAmount': 'sum',
    'InvoiceDate': ['min', 'max'],
    'Country': 'first'
}).reset_index()

In [67]:
# Rename columns
customer_df.columns = ['CustomerID', 'NumOrders', 'TotalQuantity', 'TotalSpent', 'FirstPurchase', 'LastPurchase', 'Country']



In [68]:
# Recency (days since last purchase)
latest_date = df['InvoiceDate'].max()
customer_df['Recency'] = (latest_date - customer_df['LastPurchase']).dt.days

# Step 5. Data Preparation for Clustering

In [70]:
# Drop non-numeric or ID columns for clustering
clustering_df = customer_df[['NumOrders', 'TotalQuantity', 'TotalSpent', 'Recency']]


# Step 6: Feature Scaling

In [72]:
scaler = StandardScaler()
scaled_features = scaler.fit_transform(clustering_df)

# Step 7 : Cleaned Data Save

In [ ]:
df.to_excel("cleaned_data.xlsx", index=False)
print("Cleaned data saved successfully to 'Online Retail Cleaned.xlsx'")